In [1]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
import os
import sys

In [2]:
base_dir = 'C:/Users/wanis'

In [3]:
train_dir = os.path.join(base_dir, 'new train')
validation_dir = os.path.join(base_dir, 'new validation')

In [4]:
batch_size = 15
epochs = 20
img_size = 224

In [5]:
import numpy as np
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import load_model

In [6]:
# from tensorflow.keras.applications.vgg16 import VGG16
# model=VGG16(include_top=False , weights="imagenet")

In [7]:
# from tensorflow.keras.optimizers import SGD

In [8]:
# pip install cv2

In [9]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
import numpy as np

In [10]:
# define cnn model
def define_model():
	# load model
	model = VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
	# model = load_model('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
#	output = Dense(5, activation='sigmoid')(class1)
	output = Dense(7, activation='softmax')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
#	opt = SGD(lr=0.001, momentum=0.9)
	opt = tf.keras.optimizers.RMSprop(lr=0.0001)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

In [11]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
	
	# plot accuracy
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')
	
	# plot loss
    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,max(plt.ylim())])
    plt.title('Training and Validation Loss')
	
	# save plot to file
    filename = 'vgg_b50e5'
    plt.savefig(filename + '_plot.png')
    plt.close()

In [12]:
# run the test harness for evaluating a model
def run_test_harness():
	# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(featurewise_center=True)
	# specify imagenet mean values for centering
	datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterator
	train_it = datagen.flow_from_directory(train_dir,
		class_mode='categorical', batch_size=batch_size, target_size=(img_size, img_size))
	validate_it = datagen.flow_from_directory(validation_dir,
		class_mode='categorical', batch_size=batch_size, target_size=(img_size, img_size))
	# fit model
	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=validate_it, validation_steps=len(validate_it), epochs=epochs, verbose=1)
	# evaluate model
	acc = model.evaluate_generator(validate_it, steps=len(validate_it), verbose=0)
	acc = np.array(acc)
	print(acc[1]*100)    
    # save model
	model.save('C:/Users/wanis/final_model_700_7_softmax_bcross_20.h5')
	print('Model saved as "final_model_700_7_softmax_bcross_20.h5"')
	# learning curves
	summarize_diagnostics(history)

In [13]:
# entry point, run the test harness
run_test_harness()

C:\Users\wanis\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Found 700 images belonging to 7 classes.
Found 140 images belonging to 7 classes.


C:\Users\wanis\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  app.launch_new_instance()


Epoch 1/2
35/35 [==============================] - 209s 6s/step - loss: 0.7634 - accuracy: 0.6843 - val_loss: 0.2627 - val_accuracy: 0.8786
Epoch 2/2
35/35 [==============================] - 214s 6s/step - loss: 0.0795 - accuracy: 0.9729 - val_loss: 0.2349 - val_accuracy: 0.8857


C:\Users\wanis\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.


88.57142925262451
Model saved as "final_model_700_7_softmax_bcross_20.h5"


In [14]:
######
## Extract Features:
# model_feature = load_model('VGG16.h5')
model_feature = VGG16(include_top=False, input_shape=(224, 224, 3))
model_feature.summary()

import pandas as pd
train_img = os.listdir(train_dir)
images = []
vgg16_feature_list = []
label = []
# print(train_img)
# exit()
for folder in train_img:
#    print(folder)
    for filename in os.listdir('new train/'+folder):
        label.append(folder)
#        print('new train/'+folder+'/'+filename)
#        im = os.path.join('train',filename)
        img_path = 'new train/'+folder+'/'+filename
        images.append(img_path)
        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        vgg16_feature = model_feature.predict(img_data)
        vgg16_feature_np = np.array(vgg16_feature)
        vgg16_feature_list.append(vgg16_feature_np.flatten())
        
vgg16_feature_list_np = np.array(vgg16_feature_list)

df_feature = pd.DataFrame(list(zip(images,vgg16_feature_list_np,label)),columns= ['img_path','feature_vector','label'])

df_feature.to_pickle('data feature 700')

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
###
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_labels():
	datagen = ImageDataGenerator(featurewise_center=True)
	
	train_it = datagen.flow_from_directory(train_dir,class_mode='categorical', batch_size=batch_size, target_size=(img_size, img_size))
#	y_true_labels = train_it.classes
	label_map = (train_it.class_indices)
	print('Class labels: ',label_map)

get_labels()

Found 700 images belonging to 7 classes.
Class labels:  {'blazer_men': 0, 'blazer_women': 1, 'dress': 2, 'hoodie': 3, 'jeans_men': 4, 'jeans_women': 5, 'shorts': 6}


In [16]:
from PIL import Image

In [17]:
###
# Resize images
path = "C:/Users/wanis/new train"
folder = os.listdir(path)

path2 = "C:/Users/wanis/v4/static/new train"

def resize():
	for one in folder:
	    dirs = os.listdir(path+'/'+one)
	    for item in dirs:
	       if os.path.isfile(path+'/'+one+'/'+item):
	           im = Image.open(path+'/'+one+'/'+item)
	           
	           imResize = im.resize((200,300), Image.ANTIALIAS)
	           imResize.save(path2+'/'+one+'/'+item, 'JPEG', quality=90)

resize()


In [18]:
# Extract feature vectors of train data

import pandas as pd

base_dir = 'C:/Users/wanis'

train_dir = os.path.join(base_dir, 'new train')
validation_dir = os.path.join(base_dir, 'new validation')

train_img = os.listdir(train_dir)
images = []
vgg16_feature_list = []
label = []

model_feature = VGG16(include_top=False, input_shape=(224, 224, 3))
# model_feature = load_model('VGG16.h5')

In [19]:
for folder in train_img:
#    print(folder)
    for filename in os.listdir(base_dir+'/new train/'+folder):
        label.append(folder)
#        print('train/'+folder+'/'+filename)
#        im = os.path.join('train',filename)
        img_path = base_dir+'/new train/'+folder+'/'+filename
        images.append(img_path)
        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        vgg16_feature = model_feature.predict(img_data)
        vgg16_feature_np = np.array(vgg16_feature)
        vgg16_feature_list.append(vgg16_feature_np.flatten())

In [20]:
vgg16_feature_list_np = np.array(vgg16_feature_list)

In [21]:
df_feature = pd.DataFrame(list(zip(images,vgg16_feature_list_np,label)),columns= ['img_path','feature_vector','label'])

In [22]:
df_feature.to_pickle('data feature 700')